# Udacity - Machine Learning Engineer Nanodegree
## Capstone Project
### Title: Development of a LSTM Network to Predict Students’ Answers on Exam Questions

### Implementation of DKT:
#### Part 1: Define constants

In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np

fn = "data/ASSISTments_skill_builder_data.csv" # Dataset path
verbose = 1 # Verbose = {0,1,2}
best_model_weights = "weights/bestmodel" # File to save the model.
log_dir = "logs" # Path to save the logs.
optimizer = "adam" # Optimizer to use
lstm_units = 100 # Number of LSTM units
batch_size = 32 # Batch size
epochs = 10 # Number of epochs to train
dropout_rate = 0.3 # Dropout rate
test_fraction = 0.2 # Portion of data to be used for testing
validation_fraction = 0.2 # Portion of training data to be used for validation

#### Part 2: Pre-processing

In [2]:
from deepkt import deepkt, data_util, metrics


dataset, length, nb_features, nb_skills = data_util.load_dataset(fn=fn,
                                                                 batch_size=batch_size,
                                                                 shuffle=True)

train_set, test_set, val_set = data_util.split_dataset(dataset=dataset,
                                                       total_size=length,
                                                       test_fraction=test_fraction,
                                                       val_fraction=validation_fraction)


set_sz = length * batch_size
test_set_sz = (set_sz * test_fraction)
val_set_sz = (set_sz - test_set_sz) * validation_fraction
train_set_sz = set_sz - test_set_sz - val_set_sz
print("============= Data Summary =============")
print("Total number of students: %d" % set_sz)
print("Training set size: %d" % train_set_sz)
print("Validation set size: %d" % val_set_sz)
print("Testing set size: %d" % test_set_sz)
print("Number of skills: %d" % nb_skills)
print("Number of features in the input: %d" % nb_features)
print("========================================")

/Users/maywzh/.pyenv/versions/3.8.6/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3338: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
============= Data Summary =============
Total number of students: 4032
Training set size: 2580
Validation set size: 645
Testing set size: 806
Number of skills: 123
Number of features in the input: 246


In [3]:
train_set

<SkipDataset shapes: ((32, None, None), (32, None, None)), types: (tf.float32, tf.float32)>

In [4]:
 df = pd.read_csv(fn)

/Users/maywzh/.pyenv/versions/3.8.6/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
df

,order_id,assignment_id,user_id,assistment_id,problem_id,original,correct,attempt_count,ms_first_response,tutor_mode,...,hint_count,hint_total,overlap_time,template_id,answer_id,answer_text,first_action,bottom_hint,opportunity,opportunity_original
0,33022537,277618,64525,33139,51424,1,1,1,32454,tutor,...,0,3,32454,30799,NaN,26,0.0,NaN,1.0,1.0
1,33022709,277618,64525,33150,51435,1,1,1,4922,tutor,...,0,3,4922,30799,NaN,55,0.0,NaN,2.0,2.0
2,35450204,220674,70363,33159,51444,1,0,2,25390,tutor,...,0,3,42000,30799,NaN,88,0.0,NaN,1.0,1.0
3,35450295,220674,70363,33110,51395,1,1,1,4859,tutor,...,0,3,4859,30059,NaN,41,0.0,NaN,2.0,2.0
4,35450311,220674,70363,33196,51481,1,0,14,19813,tutor,...,3,4,124564,30060,NaN,65,0.0,0.0,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525529,38214014,291495,96299,57830,108976,0,1,1,38234,tutor,...,0,2,38234,55692,200260.0,NaN,0.0,NaN,54.0,NaN
525530,38214016,291495,96299,57843,109015,0,1,1,6500,tutor,...,0,0,6500,55693,200299.0,NaN,0.0,NaN,55.0,NaN
525531,38214195,291503,96299,34577,54060,0,1,1,18344,tutor,...,0,3,18344,30677,NaN,0.8,0.0,NaN,56.0,NaN
525532,38214196,291503,96299,34577,54061,0,1,1,12922,tutor,...,0,2,12922,30677,NaN,-6,0.0,NaN,57.0,NaN


In [6]:
if "skill_id" not in df.columns:
    raise KeyError(f"The column 'skill_id' was not found on {fn}")
if "correct" not in df.columns:
    raise KeyError(f"The column 'correct' was not found on {fn}")
if "user_id" not in df.columns:
    raise KeyError(f"The column 'user_id' was not found on {fn}")

In [8]:
if not (df['correct'].isin([0, 1])).all():
        raise KeyError(f"The values of the column 'correct' must be 0 or 1.")

In [9]:
  df.dropna(subset=['skill_id'], inplace=True)

In [11]:
df = df.groupby('user_id').filter(lambda q: len(q) > 1).copy()

In [12]:
df['skill'], _ = pd.factorize(df['skill_id'], sort=True)

In [23]:
df

,order_id,assignment_id,user_id,assistment_id,problem_id,original,correct,attempt_count,ms_first_response,tutor_mode,...,hint_total,overlap_time,template_id,answer_id,answer_text,first_action,bottom_hint,opportunity,opportunity_original,skill
0,33022537,277618,64525,33139,51424,1,1,1,32454,tutor,...,3,32454,30799,NaN,26,0.0,NaN,1.0,1.0,0
1,33022709,277618,64525,33150,51435,1,1,1,4922,tutor,...,3,4922,30799,NaN,55,0.0,NaN,2.0,2.0,0
2,35450204,220674,70363,33159,51444,1,0,2,25390,tutor,...,3,42000,30799,NaN,88,0.0,NaN,1.0,1.0,0
3,35450295,220674,70363,33110,51395,1,1,1,4859,tutor,...,3,4859,30059,NaN,41,0.0,NaN,2.0,2.0,0
4,35450311,220674,70363,33196,51481,1,0,14,19813,tutor,...,4,124564,30060,NaN,65,0.0,0.0,3.0,3.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459202,33150328,277900,85730,69050,128450,1,0,2,51594,tutor,...,4,130875,59998,NaN,0,0.0,NaN,12.0,12.0,122
459203,33150408,277900,85730,69042,128442,1,0,2,51016,tutor,...,4,102125,59998,NaN,03/jul,0.0,NaN,13.0,13.0,122
459204,33150487,277900,85730,69056,128456,1,1,1,61547,tutor,...,4,61547,59998,NaN,2-May,0.0,NaN,14.0,14.0,122
459205,33150779,277900,85730,69046,128446,1,1,1,53890,tutor,...,4,53890,59998,NaN,10/-6,0.0,NaN,15.0,15.0,122


In [20]:
df['skill_with_answer'] = df['skill'] * 2 + df['correct']

In [13]:
df

,order_id,assignment_id,user_id,assistment_id,problem_id,original,correct,attempt_count,ms_first_response,tutor_mode,...,hint_total,overlap_time,template_id,answer_id,answer_text,first_action,bottom_hint,opportunity,opportunity_original,skill
0,33022537,277618,64525,33139,51424,1,1,1,32454,tutor,...,3,32454,30799,NaN,26,0.0,NaN,1.0,1.0,0
1,33022709,277618,64525,33150,51435,1,1,1,4922,tutor,...,3,4922,30799,NaN,55,0.0,NaN,2.0,2.0,0
2,35450204,220674,70363,33159,51444,1,0,2,25390,tutor,...,3,42000,30799,NaN,88,0.0,NaN,1.0,1.0,0
3,35450295,220674,70363,33110,51395,1,1,1,4859,tutor,...,3,4859,30059,NaN,41,0.0,NaN,2.0,2.0,0
4,35450311,220674,70363,33196,51481,1,0,14,19813,tutor,...,4,124564,30060,NaN,65,0.0,0.0,3.0,3.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459202,33150328,277900,85730,69050,128450,1,0,2,51594,tutor,...,4,130875,59998,NaN,0,0.0,NaN,12.0,12.0,122
459203,33150408,277900,85730,69042,128442,1,0,2,51016,tutor,...,4,102125,59998,NaN,03/jul,0.0,NaN,13.0,13.0,122
459204,33150487,277900,85730,69056,128456,1,1,1,61547,tutor,...,4,61547,59998,NaN,2-May,0.0,NaN,14.0,14.0,122
459205,33150779,277900,85730,69046,128446,1,1,1,53890,tutor,...,4,53890,59998,NaN,10/-6,0.0,NaN,15.0,15.0,122


In [21]:
seq = df.groupby('user_id').apply(
        lambda r: (
            r['skill_with_answer'].values[:-1],
            r['skill'].values[1:],
            r['correct'].values[1:],
        )
    )

In [28]:
seq[14]

(array([ 2,  3,  2,  2,  2,  2,  2,  2,  3,  2,  3,  2, 18, 18, 18, 18, 23,
        23, 23, 48, 49, 48, 48, 48, 48, 48, 49, 49, 48, 62, 62, 63, 62, 63,
        62, 88, 89, 88, 88, 88, 88, 96, 96, 98]),
 array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  9,  9,  9,  9, 11, 11,
        11, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 31, 31, 31, 31, 31, 31,
        44, 44, 44, 44, 44, 44, 48, 48, 49, 49]),
 array([1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0,
        0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]))

In [22]:
    nb_users = len(seq)

In [30]:
nb_users

4047

In [23]:
dataset = tf.data.Dataset.from_generator(
        generator=lambda: seq,
        output_types=(tf.int32, tf.int32, tf.float32)
    )

In [24]:
dataset

<FlatMapDataset shapes: (<unknown>, <unknown>, <unknown>), types: (tf.int32, tf.int32, tf.float32)>

In [25]:
dataset

<FlatMapDataset shapes: (<unknown>, <unknown>, <unknown>), types: (tf.int32, tf.int32, tf.float32)>

In [26]:
dataset = dataset.shuffle(buffer_size=nb_users)

In [27]:
dataset = dataset.map(
    lambda feat, skill, label: (
        tf.one_hot(feat, depth=features_depth),
        tf.concat(
            values=[
                tf.one_hot(skill, depth=skill_depth),
                tf.expand_dims(label, -1)
            ],
            axis=-1
        )
    )
)

NameError: in user code:

    <ipython-input-27-c2ba6e5b2828>:2 None  *
        )

    NameError: name 'features_depth' is not defined


#### Part 3: Building the model

In [3]:
student_model = deepkt.DKTModel(
                        nb_features=nb_features,
                        nb_skills=nb_skills,
                        hidden_units=lstm_units,
                        dropout_rate=dropout_rate)

student_model.compile(
        optimizer=optimizer,
        metrics=[
            metrics.BinaryAccuracy(),
            metrics.AUC(),
            metrics.Precision(),
            metrics.Recall()
        ])

student_model.summary()

Model: "DKTModel"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, None, 246)]       0         
_________________________________________________________________
masking (Masking)            (None, None, 246)         0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 100)         138800    
_________________________________________________________________
outputs (TimeDistributed)    (None, None, 123)         12423     
Total params: 151,223
Trainable params: 151,223
Non-trainable params: 0
_________________________________________________________________


#### Part 4: Train the Model

In [4]:
import tensorflow as tf
history = student_model.fit(dataset=train_set,
                            epochs=epochs,
                            verbose=verbose,
                            validation_data=val_set,
                            callbacks=[ 
                                tf.keras.callbacks.CSVLogger(f"{log_dir}/train.log"),
                                tf.keras.callbacks.ModelCheckpoint(best_model_weights,
                                                                   save_best_only=True,
                                                                   save_weights_only=True),
                                tf.keras.callbacks.TensorBoard(log_dir=log_dir)
        ])

Epoch 1/10
83/83 [==============================] - 12s 146ms/step - loss: 0.0657 - binary_accuracy: 0.7288 - auc: 0.7076 - precision: 0.7478 - recall: 0.9222 - val_loss: 0.0000e+00 - val_binary_accuracy: 0.0000e+00 - val_auc: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/10
83/83 [==============================] - 9s 108ms/step - loss: 0.0557 - binary_accuracy: 0.7720 - auc: 0.8120 - precision: 0.7765 - recall: 0.9440 - val_loss: 0.0666 - val_binary_accuracy: 0.7878 - val_auc: 0.8344 - val_precision: 0.7908 - val_recall: 0.9487
Epoch 3/10
83/83 [==============================] - 9s 108ms/step - loss: 0.0612 - binary_accuracy: 0.7788 - auc: 0.8233 - precision: 0.7862 - recall: 0.9348 - val_loss: 0.0607 - val_binary_accuracy: 0.7715 - val_auc: 0.8289 - val_precision: 0.7733 - val_recall: 0.9310
Epoch 4/10
83/83 [==============================] - 9s 103ms/step - loss: 0.0596 - binary_accuracy: 0.7877 - auc: 0.8315 - precision: 0.7991 - recall: 0.9319 - val_loss:

#### Part 5: Load the Model with the Best Validation Loss

In [5]:
student_model.load_weights(best_model_weights)

#### Part 6: Test the Model

In [6]:
result = student_model.evaluate(test_set, verbose=verbose)

     26/Unknown - 2s 67ms/step - loss: 0.0529 - binary_accuracy: 0.8071 - auc: 0.8599 - precision: 0.8176 - recall: 0.9329